In [17]:
import torch
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as prf

from forward_step import ComputeLossVAE
from model import DAGMM_VAE

def eval(args, model, dataloaders, device, sub=20):
    """Evaluate the DAGMM-VAE model with GMM energy scoring."""
    train_loader, test_loader = dataloaders
    model.eval()
    print('Evaluating DAGMM-VAE...')

    # Use ComputeLossVAE only for its compute_params and compute_energy methods
    compute = ComputeLossVAE(
        lambda_energy=args.lambda_energy,
        lambda_cov=args.lambda_cov,
        lambda_kl=args.lambda_kl,
        device=device,
        n_gmm=args.n_gmm
    )

    # 1) Estimate GMM parameters on training (clean) data
    with torch.no_grad():
        N = 0
        gamma_sum = 0
        mu_sum = 0
        cov_sum = 0
        for x, _ in train_loader:
            x = x.float().to(device)
            outputs = model(x)
            z = outputs['z']
            gamma = outputs['gamma']

            phi_batch, mu_batch, cov_batch = compute.compute_params(z, gamma)
            batch_gamma_sum = gamma.sum(dim=0)

            gamma_sum += batch_gamma_sum
            mu_sum    += mu_batch * batch_gamma_sum.unsqueeze(-1)
            cov_sum   += cov_batch * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1)
            N += x.size(0)

        phi = gamma_sum / N
        mu  = mu_sum / gamma_sum.unsqueeze(-1)
        cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)

    # 2) Compute energy scores for train and test
    def get_scores(loader):
        scores, labels = [], []
        with torch.no_grad():
            for x, y in loader:
                x = x.float().to(device)
                outputs = model(x)
                z = outputs['z']
                gamma = outputs['gamma']
                energy, _ = compute.compute_energy(z, gamma, phi=phi, mu=mu, cov=cov, sample_mean=False)
                scores.append(energy.cpu())
                labels.append(y)
        return torch.cat(scores).numpy(), torch.cat(labels).numpy()

    energy_train, labels_train = get_scores(train_loader)
    energy_test,  labels_test  = get_scores(test_loader)

    # Combine for threshold and AUC
    all_scores = np.concatenate([energy_train, energy_test])
    all_labels = np.concatenate([labels_train,  labels_test])

    # Set threshold (e.g., top 20% anomalies)
    thresh = np.percentile(all_scores, 100-sub)
    preds = (energy_test > thresh).astype(int)

    precision, recall, f1, _ = prf(labels_test, preds, average='binary')
    auc = roc_auc_score(all_labels, all_scores)

    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    print(f"ROC AUC: {auc*100:.2f}%")

    return all_labels, all_scores


In [18]:
import numpy as np
import torch

import matplotlib.pyplot as plt
import pandas as pd

from preprocess import get_KDDCup99
from train import TrainerDAGMMVAE
# from test import eval


In [19]:

class Args:
    num_epochs     = 100
    patience       = 50
    lr             = 1e-4
    lr_milestones  = [50]
    batch_size     = 1024
    latent_dim     = 1
    n_gmm          = 4
    lambda_energy  = 0.1
    lambda_cov     = 0.005
    lambda_kl      = 0.0

args   = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data   = get_KDDCup99(args)
vae_trainer = TrainerDAGMMVAE(args, data, device)
vae_trainer.train()


cuda
198371/198371: [===============================>] - ETA 0.1ss
Training DAGMM-VAE... Epoch: 0, Loss: 0.188
198371/198371: [===============================>] - ETA 0.1ss
Training DAGMM-VAE... Epoch: 1, Loss: 0.153
198371/198371: [===============================>] - ETA 0.1ss
Training DAGMM-VAE... Epoch: 2, Loss: 0.142
198371/198371: [===============================>] - ETA 0.1ss
Training DAGMM-VAE... Epoch: 3, Loss: 0.141
198371/198371: [===============================>] - ETA 0.1ss
Training DAGMM-VAE... Epoch: 4, Loss: 0.136
198371/198371: [===============================>] - ETA 0.1ss
Training DAGMM-VAE... Epoch: 5, Loss: 0.120
198371/198371: [===============================>] - ETA 0.0s
Training DAGMM-VAE... Epoch: 6, Loss: 0.112
198371/198371: [===============================>] - ETA 0.1ss
Training DAGMM-VAE... Epoch: 7, Loss: 0.109
198371/198371: [===============================>] - ETA 0.0s
Training DAGMM-VAE... Epoch: 8, Loss: 0.107
198371/198371: [===========================

KeyboardInterrupt: 

In [ ]:

# 2) Evaluate at multiple thresholds
sub_percentiles = [10, 20, 30, 40, 50, 60]
for sub in sub_percentiles:
    print(f"\n=== Threshold: top {sub}% anomalies ===")
    y_true, scores = eval(args, vae_trainer.model, data, device, sub)
    # if you want to plot the score distribution:
    # plt.hist(scores, bins=50); plt.title(f"Score hist at {sub}%"); plt.show()



=== Threshold: top 10% anomalies ===
Evaluating DAGMM-VAE...
Precision: 0.2387, Recall: 0.0688, F1: 0.1069
ROC AUC: 47.90%

=== Threshold: top 20% anomalies ===
Evaluating DAGMM-VAE...
Precision: 0.2898, Recall: 0.1733, F1: 0.2169
ROC AUC: 47.91%

=== Threshold: top 30% anomalies ===
Evaluating DAGMM-VAE...


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def plot_inlier_outlier_kde(labels, scores, model_name='DAGMM', sub=None):
    """
    Plot KDEs of inlier (label=0) vs outlier (label=1) score distributions.

    Args:
        labels (array-like): 0 for inliers, 1 for outliers
        scores (array-like): anomaly scores
        model_name (str): Used in the title
        sub (int, optional): percentile threshold used, if any
    """
    # Split scores
    scores_in  = scores[np.where(labels == 0)[0]]
    scores_out = scores[np.where(labels == 1)[0]]

    # Make DataFrames
    df_in  = pd.DataFrame(scores_in,  columns=['Inlier'])
    df_out = pd.DataFrame(scores_out, columns=['Outlier'])

    # Plot
    fig, ax = plt.subplots(figsize=(8, 4))
    df_in .plot.kde(ax=ax, legend=True)
    df_out.plot.kde(ax=ax, legend=True)

    # Title & grids
    title = f'{model_name} Inlier vs Outlier KDE'
    if sub is not None:
        title += f' (top {sub}% threshold)'
    ax.set_title(title)
    ax.grid(axis='x', linestyle='--', alpha=0.5)
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    ax.set_xlabel('Anomaly Score')
    plt.tight_layout()
    plt.show()
 

In [ ]:
labels_vae, scores_vae = eval(vae_trainer.model, data, device, args.n_gmm, sub=30)
plot_inlier_outlier_kde(labels_vae, scores_vae, model_name='DAGMM‑VAE', sub=20)
